 ## Skewed Data - Inference
 In one example, a skewed distribution was transformed prior to conducting the analysis with a regression tree. Another approach could be to use a more robust statistic such as the median. One limitation of the median, is that a linear regression model as we have covered so far, does not allow you to fit the model while using the median.

In [ ]:
library(tidyverse)
library(ggformula)
library(mosaic)
library(broom)

theme_set(theme_bw(base_size = 18))

college_score <- read_csv("https://raw.githubusercontent.com/lebebr01/statthink/master/data-raw/College-scorecard-4143.csv", guess_max = 10000)
head(college_score)

adm_model <- lm(adm_rate ~ preddeg, data = college_score)
coef(adm_model)

 Prior to doing the median, we can bootstrap the mean difference from the model above.

In [ ]:
resample_admrate <- function(...) {
  college_resample <- college_score |>
    sample_n(nrow(college_score), replace = TRUE)

  lm(adm_rate ~ preddeg, data = college_resample) |>
    tidy() |>
    select(term, estimate)
}

resample_admrate()

In [ ]:
admrate_coef <- map_dfr(1:10000, resample_admrate)

admrate_coef |>
  gf_density(~ estimate) |> 
  gf_facet_wrap(~ term, scales = 'free_x')

 ## Bootstrap Median
 The bootstrap for the median will take much of a similar process as before, the major difference being that a model will not be fitted. Instead, we will compute statistics for the median of each group, take differences of the median to represent the median difference between the groups and then replicate.

  1. Resample the observed data available, with replacement
  2. Estimate median for each group.
  3. Calculate median difference between the groups
  4. Repeat steps 1 - 3 many times
  5. Explore the distribution of median differences from the many resampled data sets.

In [ ]:
resample_admrate_median <- function(...) {
  college_resample <- college_score |>
    sample_n(nrow(college_score), replace = TRUE)

  med_est <- college_resample |>
    df_stats(adm_rate ~ preddeg, median) |>
    spread(preddeg, median)

  names(med_est)[2:4] <- c("Associate", "Bachelor", "Certificate")
  
  med_est |> 
    mutate(bachelor_associate = Bachelor - Associate,
           certificate_associate = Certificate - Associate,
           bachelor_certificate = Bachelor - Certificate) |>
    gather(Associate:bachelor_certificate, 
                 key = "Term", 
                 value = "Median_Difference")
}

resample_admrate_median()

In [ ]:
admrate_median <- map_dfr(1:10000, resample_admrate_median)

admrate_median |>
    filter(Term %in% c('bachelor_associate', 'certificate_associate', 'bachelor_certificate')) |>
    gf_density(~ Median_Difference) |> 
    gf_facet_wrap(~ Term, scales = 'free_x')